In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# import chart_studio as py
import cufflinks as cf
cf.go_offline(connected=True)
from scipy import optimize 
from datetime import datetime
from dateutil.relativedelta import relativedelta
%matplotlib inline

try:
  import pandas_datareader.data as web
except:
  !pip install pip pandas-datareader
  import pandas_datareader.data as web

In [ ]:
class Asset():
  def __init__(self, ticker, name, start, end):
    self.ticker = ticker
    self.name = name
    
    self.start = start
    self.end = end

    # 데이터 불러오기
    self.data = web.get_data_yahoo(self.ticker, self.start, self.end)
    self.date = self.data.index

    self.ratio = 0
    self.price = None
    self.change = None

  def put_price_change(self):
    self.price = self.data['Adj Close']
    self.change = self.price.pct_change().to_list()

  def get_date(self):
    return self.date

  def get_change(self):
    return self.change

  def get_name(self):
    return self.name

In [ ]:
class AWF_Asset(Asset):
    def __init__(self, equity_ticker_list, inl_ticker_list, corp_ticker_list,
                 bond_ticker_list, emd_ticker_list, comm_ticker_list, start, end):
        self.start = start
        self.end = end
        self.result = []
        self.asset_list = []
        self.total_ticker_dict = {
            "equity_ticker_list" : equity_ticker_list,
            "inl_ticker_list" : inl_ticker_list,
            "corp_ticker_list" : corp_ticker_list,
            "bond_ticker_list" : bond_ticker_list,
            "emd_ticker_list" : emd_ticker_list,
            "comm_ticker_list" : comm_ticker_list
        }
        self.result_dict = {
            "equity" : [],
            "inl" : [],
            "corp" : [],
            "bond" : [],
            "emd" : [],
            "comm" : []
        }
        
    def create_dict(self):
        for t_key in self.total_ticker_dict:
            for stock in self.total_ticker_dict[t_key]:
                asset = Asset(stock, stock, self.start, self.end)
                self.asset_list.append(asset)
                value = asset.data["Adj Close"].pct_change().dropna()
                
                key = t_key.split("_")[0]
                self.result_dict[key].append(value)
                
    def calculate_value(self):
        for label in self.result_dict:
            value = self.result_dict[label][0]
            
            for i in range(len(self.result_dict[label]) - 1):
                value += self.result_dict[label][i]
            
            avg = value / len(self.result_dict[label])
            self.result.append(avg)
        return self.result
    
    def get_value(self):
        self.create_dict()
        return self.calculate_value(), self.total_ticker_dict

In [ ]:
class AllWeatherPortfolio(Asset):
    
    def __init__(self, dictionary, ticker):
        self.result = dictionary
        self.total_ticker_dict = ticker
   
    def merge(self):
        port = pd.concat(self.result, axis =1 )
        port.columns = ['equity', 'infl', 'corp', 'bond', 'emd', 'comm']
        asset_num = len(port.columns)

        port_gr_r = (self.result[0] + self.result[2] + self.result[4] + self.result[5])/4
        port_gr_f = (self.result[1] + self.result[4] + self.result[5])/3
        port_inf_r = (self.result[1] + self.result[3])/2
        port_inf_f = (self.result[0] + self.result[3])/2

        portfolio = pd.concat([port_gr_r, port_gr_f, port_inf_r, port_inf_f], axis =1 )
        portfolio.columns = ['gr_r', 'gr_f', 'inf_r', 'inf_f' ]
        self.covmat = portfolio.cov()

    def RiskParity_objective(self, x) :
        variance = x.T @ self.covmat @ x
        sigma = variance ** 0.5
        mrc = 1/sigma * (self.covmat @ x)
        rc = x * mrc  
        rc = rc/rc.sum() 
  
        a = pd.DataFrame(index = range(len(x)), columns = range(len(x)))
        for i in range (0, len(x)):
            for j in range(0, len(x)):
              a[i][j]  = rc[i] - rc[j]
  
        sum_risk_diffs_squared = np.sum(np.square(np.ravel(a)))
        return sum_risk_diffs_squared


    def weight_sum_constraint(self, x):
        result = x.sum() - 1.0
        return result

    def weight_longonly(self, x):
        return x

    def RiskParity(self):
        x0 = np.repeat(1 / self.covmat.shape[1], self.covmat.shape[1])
        con1 = {'type' : 'eq', 'fun' : self.weight_sum_constraint}
        con2 = {'type': 'ineq', 'fun' : self.weight_longonly}  
        constraint =  ([con1, con2])
        options = {'ftol' : 1e-20, }
        result = optimize.minimize(fun = self.RiskParity_objective,
                    x0 = x0,
                    method = 'SLSQP',
                    constraints = constraint,
                    options = options)
        return result.x


    def weight(self, wt_erc):
        weight = [] 
        
        for label in self.total_ticker_dict:
            if label == "equity_ticker_list":
                length = len(self.total_ticker_dict[label])
                for i in range(length):
                    weight.append(wt_erc[0]/(4*length) + wt_erc[3]/(2*length))              
            elif label == "inl_ticker_list":
                weight.append(wt_erc[1]/3 +  wt_erc[2]/2)            
            elif label == "corp_ticker_list":
                weight.append(wt_erc[0]/4)
            elif label == "bond_ticker_list":
                length = len(self.total_ticker_dict[label])
                for i in range(length):
                    weight.append(wt_erc[2]/(2*length) + wt_erc[3]/(2*length))               
            elif label == "emd_ticker_list":
                length = len(self.total_ticker_dict[label])
                for i in range(length):
                    weight.append(wt_erc[0]/(4*length) + wt_erc[1]/(2*length))
            elif label == "comm_ticker_list":
                for i in range(length):
                    weight.append(wt_erc[0]/(4*length) + wt_erc[1]/(2*length))

        return weight

    def awf(self):
        self.merge()
        wt_erc = self.RiskParity()
        x = self.weight(wt_erc)

        return x


In [ ]:
equity = ['VTI']
bond = ['EDV']
comm = ['IAU', 'GSG']

awf_asset = AWF_Asset(equity, infl, corp, bond, emd, comm, '2008-01-29', '2020-11-30')
awf_ret, awf_ticker = awf_asset.get_value()

awf_asset = AllWeatherPortfolio(awf_ret, awf_ticker)
awf_weight = awf_asset.awf()
awf_portfolio = Portfolio('All Weather', awf_asset.asset_list, awf_weight, 1000, 3, 
                          awf_dict=awf_asset.result_dict, awf_ticker=awf_asset.total_ticker_dict)
# awf_backtest_df = awf_portfolio.backtest()
# awf_backtest_result_df = awf_portfolio.backtest_result()
# awf_annual_return_df = awf_portfolio.periodic_result('annual')
# awf_monthly_return_df = awf_portfolio.periodic_result('monthly')
# awf_summary = awf_portfolio.get_summary()
# awf_summary

[0.08660932849570702,
 0.08660932849570702,
 0.22438436648195126,
 0.06013642106085254,
 0.2529454914809484,
 0.09345904372909954,
 0.09345904372909954,
 0.09345904372909954,
 0.09345904372909954]

추후 구현할 내용

Input : 체크박스 형태로 asset 받아옴

Output : Drawdown , Balance, Cummulative Return, 
+ 월별 수익률

+ Factor Model과 회귀분석 진행후 결과값 시각화

